# Preprocessing
This notebook contains all code for the preprocessing of the KDD Cup 98 datasets.
* Splits into learning and test
* Learns the transformation pipeline on the learning dataset for future use
* Prepares the data for model fitting

This will be done with scikit-learn's transforming framework in order to ensure all transformations are applied identically on training, test and validation datasets.


The transformations are 'learned' on the training dataset and then applied to the test dataset and new data later on.

In [6]:
%load_ext autoreload

In [3]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

os.chdir("../") # Change to project root
from util.data_loader import KDD98DataLoader
from kdd98.transformers import *

In [8]:
# plto config
%matplotlib inline
sns.set(color_codes=True)
sns.set_style('ticks')
plt.rcParams['figure.figsize'] = [40, 40]

## Loading the learning dataset


Set working directory to main code folder

In [11]:
%autoreload 2 # automatically reloads modules
lrn = dl.KDD98DataLoader("cup98LRN.txt")
learning_raw = lrn.get_dataset()

## Splitting into training- and test dataset

Before applying any transformations, the dataset will be split 80/20 into a learning and test set.

Let's first look at feature TARGET_B, which describes whether a person has donated or not:

In [19]:
learning_raw.TARGET_B.value_counts(normalize=True) # 5 % of recipients have donated.

0    0.949241
1    0.050759
Name: TARGET_B, dtype: float64

We want to preserve this ratio in the split datasets. scikit-learn provides a class for achieving this.

In [21]:
from config import App
seed = App.config("random_seed")
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, train_size=0.8, random_state=seed)
for learn_index, test_index in splitter.split(learning_raw, learning_raw.TARGET_B.astype('int')):
    l_i = learn_index
    t_i = test_index
    kdd_learn = learning_raw.iloc[learn_index]
    kdd_test = learning_raw.iloc[test_index]
    

Now, check that the two sets are really disjoint

In [22]:
set(kdd_learn.index).intersection(kdd_test.index)

set()

Check the frequencies of the donors in the sets:

In [23]:
kdd_learn['TARGET_B'].value_counts()/len(kdd_learn.index)

0    0.949246
1    0.050754
Name: TARGET_B, dtype: float64

In [24]:
kdd_test['TARGET_B'].value_counts()/len(kdd_test.index)

0    0.949222
1    0.050778
Name: TARGET_B, dtype: float64

## Separating features and label

First, we separate the features from the labels. We will also remove the label "TARGET_B", which is an indicator variable for donors that is no longer of interest

**All preprocessing is performed on *kdd_learn_feat***

In [26]:
kdd_learn_feat = kdd_learn.drop(['TARGET_B', 'TARGET_D'],axis=1)
kdd_learn_label = kdd_learn['TARGET_D'].copy()

## Exploring strategies for specific feature types

* Noisy data: Correction of data entry / formatting errors
    - These errors must be corrected without excluding the records in question
* Missing data: Has to be inferred from known values
    - (e.g., mean, median, mode, a modeled value).
    - One exception to this rule is the attributes containing 99.5 percent or more missings. These are to be dropped
* Sparse data: Events actually represented in given data make only a very small subset of the event space are to be dropped
* Constant values are to be dropped

### Constant and Sparse Features

Features where only one value is present and those where the majority is empty are to be dropped.


In [63]:
v = learning_raw.var()
v[v <= 1e-5]

ADATE_5     0.0
ADATE_15    0.0
dtype: float64

In [69]:
for f in kdd_learn_feat.columns:
    if len(kdd_learn_feat[f].unique()) == 1:
        print(f+" has elements: "+ str(kdd_learn_feat[f].unique()))

RFA_2R has elements: [L]
Categories (1, object): [L]


In [ ]:
sparse_features = []
for column in learning:
    top_freq = learning[column].value_counts(normalize=True).iloc[0]
    if top_freq > 0.995:
        sparse_features.append(column)
        print(column+" has a top frequency of: " + str(top_freq))
        print(learning[column].value_counts(normalize=True))

In [65]:
set(kdd_learn_feat.ADATE_5.dropna())

{9604.0}

In [66]:
set(kdd_learn_feat.ADATE_15.dropna())

{9504.0}

### Boolean features

From the dataset dictionary, a number of boolean features could be identified. These are:

In [27]:
print(dl.boolean_features)

['MAILCODE', 'NOEXCH', 'RECINHSE', 'RECP3', 'RECPGVG', 'AGEFLAG', 'HOMEOWNR', 'MAJOR', 'COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES', 'PEPSTRFL', 'TARGET_B', 'HPHONE_D']


We will now deal with these features:

### Numerical features

In [ ]:
numerical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

### Categorical features

In [ ]:
categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(categories="auto"))
    
])

### Remaining object features

In [ ]:
objects = learning_raw.select_dtypes(include='object').columns
print(objects)

In [ ]:
for f in objects:
    print(f+": "+learning_raw[f].unique())

These are two types:

* ZIP: Malformed zip codes. Some have a dash at the end, which has to be removed.
* Multibyte values. These can be extracted into separate features bytewise. However, this is done in feature extraction later on

## Preprocessing Pipeline

It is now time to construct the preprocessing pipeline. A set of transforming operations is concatenated to a sequence of operations. This pipeline is the learned on the learning dataset. All transformations to the learning dataset will then later be applied to the test dataset and to new data.

In [87]:
numerical_feats = list(kdd_learn_feat.select_dtypes(include=np.number).columns)
categorical_feats = list(kdd_learn_feat.select_dtypes(include=np.number).columns)

In [90]:
column_transformers = ColumnTransformer([
    ("bool_x_bl",
    BooleanFeatureRecode(value_map={'true': 'X', 'false': ' '}),
    ['PEPSTRFL', 'NOEXCH', 'MAJOR', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP']
    ),
    ("bool_y_n",
     BooleanFeatureRecode(value_map={'true': 'Y', 'false': 'N'}),
     ['COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS','CDPLAY', 'STEREO',
      'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN',  'BOATS', 'WALKER', 'KIDSTUFF',
      'CARDS', 'PLATES']
    ),
    ("bool_e_i",
     BooleanFeatureRecode(value_map={'true': "E", 'false': 'I'}),
     ['AGEFLAG']
    ),
    ("bool_h_u",
     BooleanFeatureRecode(value_map={'true': "H", 'false': 'U'}),
     ['HOMEOWNR']),
    ("bool_b_bl",
     BooleanFeatureRecode(value_map={'true': 'B', 'false': ' '}),
     ['MAILCODE']
    ),
    ("bool_1_0",
     BooleanFeatureRecode(value_map={'true': '1', 'false': '0'}),
     ['HPHONE_D']
    ),
    ("zipcode_trim",
     ZipCodeFormatter(),
     ['ZIP']
    ),
    ("numerical_features",
     numerical_pipe,
     numerical_feats
    ),
    ("categorical_features",
     categorical_pipe,
     categorical_feats
    )
])

## EDA

Interests and donations

In [ ]:
data = learning_raw.loc[:,dl.interest_features+["TARGET_D"]].fillna(0)
interests = pd.melt(data,value_vars=dl.interest_features, value_name="Interest")
data.head()

Features with constant values:

In [ ]:
learning_raw.nunique(axis=1)

### Individual feature properties

Value range, distribution, outliers

### Correlations

-> Product moment covariance

In [ ]:
# calculate the correlation matrix
corr = learning_raw.drop(['TARGET_B','TARGET_D'],axis=1).corr()

In [ ]:
# plot the heatmap
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 12))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.8, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

### Target variable (labels)

In [ ]:
%matplotlib inline
sns.catplot(x="WEALTH2", y="TARGET_D", hue="MAJOR",
            kind="violin", inner="stick", split=True,
            palette="pastel", data=learning);

In [ ]:
sns.catplot(x="CLUSTER", y="TARGET_D", kind="box", data=learning);

In [ ]:
%matplotlib inline
sns.distplot(learning.loc[learning.TARGET_D > 0.0, 'TARGET_D'], bins=50, kde=False, rug=True);

### US census data

In [ ]:
us_census = ["POP901", "POP902", "POP903", "POP90C1", "POP90C2", "POP90C3", "POP90C4", "POP90C5", "ETH1", "ETH2", "ETH3", "ETH4", "ETH5", "ETH6", "ETH7", "ETH8", "ETH9", "ETH10", "ETH11", "ETH12", "ETH13", "ETH14", "ETH15", "ETH16", "AGE901", "AGE902", "AGE903", "AGE904", "AGE905", "AGE906", "AGE907", "CHIL1", "CHIL2", "CHIL3", "AGEC1", "AGEC2", "AGEC3", "AGEC4", "AGEC5", "AGEC6", "AGEC7", "CHILC1", "CHILC2", "CHILC3", "CHILC4", "CHILC5", "HHAGE1", "HHAGE2", "HHAGE3", "HHN1", "HHN2", "HHN3", "HHN4", "HHN5", "HHN6", "MARR1", "MARR2", "MARR3", "MARR4", "HHP1", "HHP2", "DW1", "DW2", "DW3", "DW4", "DW5", "DW6", "DW7", "DW8", "DW9", "HV1", "HV2", "HV3", "HV4", "HU1", "HU2", "HU3", "HU4", "HU5", "HHD1", "HHD2", "HHD3", "HHD4", "HHD5", "HHD6", "HHD7", "HHD8", "HHD9", "HHD10", "HHD11", "HHD12", "ETHC1", "ETHC2", "ETHC3", "ETHC4", "ETHC5", "ETHC6", "HVP1", "HVP2", "HVP3", "HVP4", "HVP5", "HVP6", "HUR1", "HUR2", "RHP1", "RHP2", "RHP3", "RHP4", "HUPA1", "HUPA2", "HUPA3", "HUPA4", "HUPA5", "HUPA6", "HUPA7", "RP1", "RP2", "RP3", "RP4", "MSA", "ADI", "DMA", "IC1", "IC2", "IC3", "IC4", "IC5", "IC6", "IC7", "IC8", "IC9", "IC10", "IC11", "IC12", "IC13", "IC14", "IC15", "IC16", "IC17", "IC18", "IC19", "IC20", "IC21", "IC22", "IC23", "HHAS1", "HHAS2", "HHAS3", "HHAS4", "MC1", "MC2", "MC3", "TPE1", "TPE2", "TPE3", "TPE4", "TPE5", "TPE6", "TPE7", "TPE8", "TPE9", "PEC1", "PEC2", "TPE10", "TPE11", "TPE12", "TPE13", "LFC1", "LFC2", "LFC3", "LFC4", "LFC5", "LFC6", "LFC7", "LFC8", "LFC9", "LFC10", "OCC1", "OCC2", "OCC3", "OCC4", "OCC5", "OCC6", "OCC7", "OCC8", "OCC9", "OCC10", "OCC11", "OCC12", "OCC13", "EIC1", "EIC2", "EIC3", "EIC4", "EIC5", "EIC6", "EIC7", "EIC8", "EIC9", "EIC10", "EIC11", "EIC12", "EIC13", "EIC14", "EIC15", "EIC16", "OEDC1", "OEDC2", "OEDC3", "OEDC4", "OEDC5", "OEDC6", "OEDC7", "EC1", "EC2", "EC3", "EC4", "EC5", "EC6", "EC7", "EC8", "SEC1", "SEC2", "SEC3", "SEC4", "SEC5", "AFC1", "AFC2", "AFC3", "AFC4", "AFC5", "AFC6", "VC1", "VC2", "VC3", "VC4", "ANC1", "ANC2", "ANC3", "ANC4", "ANC5", "ANC6", "ANC7", "ANC8", "ANC9", "ANC10", "ANC11", "ANC12", "ANC13", "ANC14", "ANC15", "POBC1", "POBC2", "LSC1", "LSC2", "LSC3", "LSC4", "VOC1", "VOC2", "VOC3", "HC1", "HC2", "HC3", "HC4", "HC5", "HC6", "HC7", "HC8", "HC9", "HC10", "HC11", "HC12", "HC13", "HC14", "HC15", "HC16", "HC17", "HC18", "HC19", "HC20", "HC21", "MHUC1", "MHUC2", "AC1", "AC2"]
len(us_census)

## Feature Selection
Meant to reduce dimensionality by selecting only features that are 'interesting enough' to be considered in order to boost performance of calculations / improve accuracy of the estimator
- By variance threshold
- Recursive Feature Elimination by Cross-Validation
- L1-based feature selection (Logistic Regression, Lasso, SVM)
- Tree-based feature selection

See [scikit-learn: feature selection](http://scikit-learn.org/stable/modules/feature_selection.html#feature-selection)

### Removing constant features (zero variance)

In [ ]:
for column in learning.columns:
        if len(learning[column].unique()) == 1:
            print(column)

### Sparse Features

In [ ]:
sparse_features = []
for column in learning:
    top_freq = learning[column].value_counts(normalize=True).iloc[0]
    if top_freq > 0.995:
        sparse_features.append(column)
        print(column+" has a top frequency of: " + str(top_freq))
        print(learning[column].value_counts(normalize=True))

In [ ]:
sparse_features

## Feature Extraction
All explanatory fields have to be numerical for the subsequent operations with scikit-learn. Here, the necessary feature extractions are performed.

See [scikit-learn: feature extraction](http://scikit-learn.org/stable/modules/feature_extraction.html)

In [ ]:
import pandas as pd

In [ ]:
symbolic_features = []
symbolic_features.append(tds.SymbolicFeatureSpreader(
    "DOMAIN", ["U", "S"])) #Urbanicity, SocioEconomicStatus
# RFA_2 is already spread out
for i in range(3, 25):
    feature = "_".join(["RFA", str(i)])
    symbolic_features.append(tds.SymbolicFeatureSpreader(
        feature, ["R", "F", "A"])) # Recency, Frequency, Amount

spread_multibyte = pd.DataFrame(index=learning_raw.index)
for f in symbolic_features:
    f.set_tidy_dataset_ref(learning_raw)
    spread_multibyte = pd.concat([spread_multibyte,f.spread(inplace=False)],axis=1)

In [ ]:
spread_multibyte.info()

# PCA

A first look at important features

In [ ]:
from sklearn import decomposition

In [ ]:
X = learning.drop(["TARGET_B","TARGET_D"],axis=1)

In [ ]:
n_comp = 3
pca = decomposition.PCA(n_components = n_comp)
pca.fit(X)
result = pd.DataFrame(pca.transform(X), columns=["PCA%i" % i for i in range(n_comp)], index=X.index)

In [ ]:
import cProfile
domain_spreader = tds.SymbolicFieldToDummies(learning,"RFA_24",["Recency", "Frequency", "Amount"])
cProfile.run('domain_spreader.spread()', sort='time')

In [ ]:
learning.head()

In [ ]:
import os
import numpy as np
import sys
os.getcwd()
proj_dir = os.path.split(os.getcwd())[0]
if proj_dir not in sys.path:
    sys.path.append(proj_dir)

In [ ]:
import eda.tidy_dataset as tds
tidy = tds.TidyDataset("cup98LRN.txt")

In [ ]:
raw = tidy.get_raw_data()

In [ ]:
spreader = tds.SymbolicFieldToDummies(
    raw, "RFA_24", ["Recency", "Frequency", "Amount"])
spreader.spread()